# Libraries

In [125]:
#!pip install humanfriendly

In [126]:
import json, requests, os, re, math, glob, secrets, datetime
import pandas as pd

from tqdm.notebook import tqdm
from urllib.parse import quote, urlencode
from humanfriendly import format_timespan

In [2]:
base_url = 'https://www.tiktok.com'
first_url = 'https://www.tiktok.com/explore'

ROWS = 200 # jumlah maksimal data yang disediakan API Endpoint
PAGES = 0

In [3]:
S = requests.Session()

base_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:130.0) Gecko/20100101 Firefox/130.0',
}

r = S.get(first_url, headers=base_headers)


In [4]:
r.text

'<!doctype html><html lang="en"><head><meta charset="UTF-8"/><meta name="viewport" content="width=device-width,initial-scale=1"/><title>TikTok - Make Your Day</title><link rel="preconnect" href="https://sf16-website-login.neutral.ttwstatic.com"><link rel="preload" as="script" href="https://sf16-website-login.neutral.ttwstatic.com/obj/tiktok_web_login_static/tiktok-infra/csp/sdk-pre/slardar.web.pre.js"><script src="https://sf16-website-login.neutral.ttwstatic.com/obj/tiktok_web_login_static/tiktok_privacy_protection_framework/loader/1.0.0.818/index.js" crossorigin="anonymous" async data-business="serverless.tiktok.desktop" data-sw="/sw.js" data-env="production" data-region="sg"></script><script id="slardar-config" type="application/json">{"bid":"tiktok_webapp","sdkUrl":"https://sf16-website-login.neutral.ttwstatic.com/obj/tiktok_web_login_static/slardar/fe/sdk-web/browser.sg.js","slardarClient":"SlardarClient","disableInit":true}</script><script src="https://sf16-website-login.neutral.t

In [5]:
with open('test_tiktok_page.html', 'w') as f:
    f.write(r.text)

In [13]:
url_to = 'https://www.tiktok.com/api/search/general/full/?WebIdLastTime=1727051634&aid=1988&app_language=en&app_name=tiktok_web&browser_language=en-US&browser_name=Mozilla&browser_online=true&browser_platform=Win32&browser_version=5.0%20%28Windows%29&channel=tiktok_web&cookie_enabled=true&data_collection_enabled=false&device_id=7417630237233694215&device_platform=web_pc&device_type=web_h264&focus_state=true&from_page=search&history_len=3&is_fullscreen=false&is_page_visible=true&keyword=python%20programming&odinId=7417630201255953415&offset=0&os=windows&priority_region=&referer=&region=ID&screen_height=768&screen_width=1366&search_source=search_sug&tz_name=Asia%2FMakassar&user_is_login=false&web_search_code=%7B%22tiktok%22%3A%7B%22client_params_x%22%3A%7B%22search_engine%22%3A%7B%22ies_mt_user_live_video_card_use_libra%22%3A1%2C%22mt_search_general_user_live_card%22%3A1%7D%7D%2C%22search_server%22%3A%7B%7D%7D%7D&webcast_language=en&msToken=mVjL8R_MWxWzfG6ARNDVNuhXISlhx51NfK2CDYYD1Vxi7H9h36tfl6jKrW212fNU0toeb0_xM2YPOBwf6TA1LALBw960sLECx6nKX4rwUxPv-6cKRDnd5P4B0OPrSgUJqnlIztGK0Oy_8TfxaWU2&X-Bogus=DFSzsIVEZ4xANyYkt6veLRSWdKKa&_signature=_02B4Z6wo00001Qi1tHgAAIDAQrY0g.KXNPUIt7DAACTg19'

In [14]:
r = S.get(url_to, headers=base_headers)

In [15]:
r.text

'{"status_code":0,"data":[{"type":1,"item":{"id":"7250355072511626539","desc":"Python Project for Beginners 👨\u200d💻🐍 #coding #programming #softwareengineer #fyp #python ","createTime":1688104851,"video":{"id":"7250355072511626539","height":1024,"width":576,"duration":30,"ratio":"540p","cover":"https://p16-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/34747fa21b9245019942129e332153e3_1688104852?lk3s=b59d6b55\\u0026x-expires=1727226000\\u0026x-signature=1O1YgnourSPrJd26PlwNDfBedPw%3D\\u0026shp=b59d6b55\\u0026shcp=-","originCover":"https://p19-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/3568887640fc416d867b7c0f9b31ccdc_1688104852?lk3s=b59d6b55\\u0026x-expires=1727226000\\u0026x-signature=XVE5Ud83anC1CbY4m6Ow3hkKvik%3D\\u0026shp=b59d6b55\\u0026shcp=-","dynamicCover":"https://p16-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/2f4560e24b2840e2ace3625881ab41d8_1688104852?lk3s=b59d6b55\\u0026x-expires=1727226000\\u0026x-signature=S4sGB2b2SpmER3wusEdIxayDOeo%3D\\u0026shp=b59d6b55\\u0026

In [17]:
len(r.json()['data'])

12

In [18]:
base_search_url = 'https://www.tiktok.com/search?q='
api_search_url = 'https://www.tiktok.com/api/search/general/full/?'

In [19]:
keyword = 'python programming'

In [21]:
r = S.get(base_search_url + quote(keyword), headers=base_headers)

In [28]:
def generate_search_id():
    timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    random_hex_length = (32-len(timestamp))//2
    random_hex = secrets.token_hex(random_hex_length).upper()
    random_id = timestamp + random_hex
    return random_id

In [58]:
def form_api_url(page = 1, keyword = 'python programming'):
    offset = (page - 1) * 12
    params = {
        'keyword': quote(keyword),
        'offset': offset,
        'search_id': generate_search_id(),
    }
    return api_search_url + urlencode(params)

In [141]:
r = S.get(form_api_url(1, 'dasar python'), headers=base_headers)

# Hasil

In [142]:
len(r.json())

10

In [143]:
if 'data' in r.json():
    print(len(r.json()['data']))

12


In [139]:
r.json()

{'status_code': 0,
 'qc': '',
 'cursor': 240,
 'has_more': 0,
 'ad_info': {},
 'extra': {'now': 1727059490000,
  'logid': '20240923024448949935B2F6F0F78F28C9',
  'fatal_item_ids': [],
  'search_request_id': '',
  'api_debug_info': None},
 'log_pb': {'impr_id': '20240923024448949935B2F6F0F78F28C9'},
 'search_nil_info': {'search_nil_type': '',
  'is_load_more': 'is_load_more',
  'search_nil_item': 'federation_empty',
  'text_type': 10},
 'global_doodle_config': {'feedback_survey': None},
 'backtrace': ''}

In [114]:
r.json()['data'][0]

{'type': 1,
 'item': {'id': '7384607728838167824',
  'desc': 'Class Object : tutorial dasar Python untuk pemula!  Jangan lupa untuk like, komen, dan subscribe untuk mendapatkan update video tutorial terbaru! #python #sisteminformasi #teknikinformatika #dataanalytics #learnpython #beginner #Pyt #BelajarCoding #b #pemrograman #TutorialPython #PythonIndonesia #pemula #pemrograman #programming #belajarprogramming #fyp',
  'createTime': 1719363900,
  'video': {'id': '7384607728838167824',
   'height': 1024,
   'width': 576,
   'duration': 141,
   'ratio': '540p',
   'cover': 'https://p16-sign-sg.tiktokcdn.com/obj/tos-alisg-p-0037/ooW8eAfQgeIA6ZrBETAIEvFEQPAvAM7ZDiD2E8?lk3s=b59d6b55&x-expires=1727229600&x-signature=Sh2LBf2xUzgcDQ8xMOVl8myg29U%3D&shp=b59d6b55&shcp=-',
   'originCover': 'https://p16-sign-sg.tiktokcdn.com/obj/tos-alisg-p-0037/4f825f5cbae1484aac2a7a68040789cc_1719363063?lk3s=b59d6b55&x-expires=1727229600&x-signature=T4iICDu4wpX1I1T13VhV0pIiQiA%3D&shp=b59d6b55&shcp=-',
   'dynami

# Ekstraksi JSON

In [127]:
temp_dict = {}
if r.json()['data'][0]['type'] == 1:
    temp_dict['id'] = r.json()['data'][0]['item']['id']
    temp_dict['desc'] = r.json()['data'][0]['item']['desc']
    temp_dict['duration'] = r.json()['data'][0]['item']['video']['duration']
    temp_dict['duration_text'] = format_timespan(r.json()['data'][0]['item']['video']['duration'])
    temp_dict['play_count'] = r.json()['data'][0]['item']['stats']['playCount']
    temp_dict['like_count'] = r.json()['data'][0]['item']['stats']['diggCount']
    temp_dict['comment_count'] = r.json()['data'][0]['item']['stats']['commentCount']
    temp_dict['bookmark_count'] = r.json()['data'][0]['item']['stats']['collectCount']
    temp_dict['share_count'] = r.json()['data'][0]['item']['stats']['shareCount']
    temp_dict['author_id'] = r.json()['data'][0]['item']['author']['id']
    temp_dict['author_username'] = '@' + r.json()['data'][0]['item']['author']['uniqueId']
    temp_dict['author_name'] = r.json()['data'][0]['item']['author']['nickname']
    temp_dict['author_following_count'] = r.json()['data'][0]['item']['authorStats']['followingCount']
    temp_dict['author_follower_count'] = r.json()['data'][0]['item']['authorStats']['followerCount']
    temp_dict['author_like_count'] = r.json()['data'][0]['item']['authorStats']['heartCount']
    temp_dict['author_post_count'] = r.json()['data'][0]['item']['authorStats']['videoCount']
    temp_dict['url_video'] = 'https://www.tiktok.com/@' + r.json()['data'][0]['item']['author']['uniqueId'] + '/video/' + r.json()['data'][0]['item']['id']
    temp_dict['url_author'] = 'https://www.tiktok.com/@' + r.json()['data'][0]['item']['author']['uniqueId']


In [128]:
temp_dict

{'id': '7384607728838167824',
 'desc': 'Class Object : tutorial dasar Python untuk pemula!  Jangan lupa untuk like, komen, dan subscribe untuk mendapatkan update video tutorial terbaru! #python #sisteminformasi #teknikinformatika #dataanalytics #learnpython #beginner #Pyt #BelajarCoding #b #pemrograman #TutorialPython #PythonIndonesia #pemula #pemrograman #programming #belajarprogramming #fyp',
 'duration': 141,
 'duration_text': '2 minutes and 21 seconds',
 'play_count': 34500,
 'like_count': 1464,
 'comment_count': 11,
 'bookmark_count': 461,
 'share_count': 29,
 'author_id': '6951383569520608258',
 'author_username': '@ketikode_',
 'author_name': 'ketikode',
 'author_following_count': 25,
 'author_follower_count': 3420,
 'author_like_count': 7944,
 'author_post_count': 91,
 'url_video': 'https://www.tiktok.com/@ketikode_/video/7384607728838167824',
 'url_author': 'https://www.tiktok.com/@ketikode_'}

# Concat the JSON

In [130]:
li = []
for d in r.json()['data']:
    temp_dict = {}
    if d['type'] == 1:
        temp_dict['id'] = d['item']['id']
        temp_dict['desc'] = d['item']['desc']
        temp_dict['duration'] = d['item']['video']['duration']
        temp_dict['duration_text'] = format_timespan(d['item']['video']['duration'])
        temp_dict['play_count'] = d['item']['stats']['playCount']
        temp_dict['like_count'] = d['item']['stats']['diggCount']
        temp_dict['comment_count'] = d['item']['stats']['commentCount']
        temp_dict['bookmark_count'] = d['item']['stats']['collectCount']
        temp_dict['share_count'] = d['item']['stats']['shareCount']
        temp_dict['author_id'] = d['item']['author']['id']
        temp_dict['author_username'] = '@' + d['item']['author']['uniqueId']
        temp_dict['author_name'] = d['item']['author']['nickname']
        temp_dict['author_following_count'] = d['item']['authorStats']['followingCount']
        temp_dict['author_follower_count'] = d['item']['authorStats']['followerCount']
        temp_dict['author_like_count'] = d['item']['authorStats']['heartCount']
        temp_dict['author_post_count'] = d['item']['authorStats']['videoCount']
        temp_dict['url_video'] = 'https://www.tiktok.com/@' + d['item']['author']['uniqueId'] + '/video/' + d['item']['id']
        temp_dict['url_author'] = 'https://www.tiktok.com/@' + d['item']['author']['uniqueId']

        li.append(temp_dict)

In [131]:
df = pd.DataFrame(li)

In [132]:
df

,id,desc,duration,duration_text,play_count,like_count,comment_count,bookmark_count,share_count,author_id,author_username,author_name,author_following_count,author_follower_count,author_like_count,author_post_count,url_video,url_author
0,7384607728838167824,Class Object : tutorial dasar Python untuk pem...,141,2 minutes and 21 seconds,34500,1464,11,461,29,6951383569520608258,@ketikode_,ketikode,25,3420,7944,91,https://www.tiktok.com/@ketikode_/video/738460...,https://www.tiktok.com/@ketikode_
1,7383870655286054149,3 Menit yang Bikin Lu Jago Python Selamanya - ...,198,3 minutes and 18 seconds,17000,1060,8,454,72,7369452789070840838,@spasialmind,spasial mind,8,2832,13200,18,https://www.tiktok.com/@spasialmind/video/7383...,https://www.tiktok.com/@spasialmind
2,7382111765322042641,Escape Character : tutorial dasar Python untuk...,201,3 minutes and 21 seconds,378,29,0,3,0,6951383569520608258,@ketikode_,ketikode,25,3420,7944,91,https://www.tiktok.com/@ketikode_/video/738211...,https://www.tiktok.com/@ketikode_
3,7299359391059299590,Belajar mengenal List pada python #pythondasar...,207,3 minutes and 27 seconds,611,21,1,5,0,6996621196604458010,@alvinadjie,Alvin | Odoo ERP Developer,643,1170,19700,101,https://www.tiktok.com/@alvinadjie/video/72993...,https://www.tiktok.com/@alvinadjie
4,7380509826603863312,Pelajari Python Dasar dengan Mudah! Apa saja y...,36,36 seconds,263,11,0,2,0,6951383569520608258,@ketikode_,ketikode,25,3420,7944,91,https://www.tiktok.com/@ketikode_/video/738050...,https://www.tiktok.com/@ketikode_
5,7383141324674878721,If Else : tutorial dasar Python untuk pemula! ...,581,9 minutes and 41 seconds,8677,419,3,115,4,6951383569520608258,@ketikode_,ketikode,25,3420,7944,91,https://www.tiktok.com/@ketikode_/video/738314...,https://www.tiktok.com/@ketikode_
6,7383846942226369809,Fnctionu : tutorial dasar Python untuk pemula!...,335,5 minutes and 35 seconds,292,29,0,8,0,6951383569520608258,@ketikode_,ketikode,25,3420,7944,91,https://www.tiktok.com/@ketikode_/video/738384...,https://www.tiktok.com/@ketikode_
7,7383507387501579528,forloop : : tutorial dasar Python untuk pemula...,540,9 minutes,280,30,1,5,0,6951383569520608258,@ketikode_,ketikode,25,3420,7944,91,https://www.tiktok.com/@ketikode_/video/738350...,https://www.tiktok.com/@ketikode_
8,7382100094142713105,Fstring : tutorial dasar Python untuk pemula! ...,169,2 minutes and 49 seconds,510,35,0,7,2,6951383569520608258,@ketikode_,ketikode,25,3420,7944,91,https://www.tiktok.com/@ketikode_/video/738210...,https://www.tiktok.com/@ketikode_
9,7383849895309954305,function bagian 2 : tutorial dasar Python untu...,250,4 minutes and 10 seconds,350,40,0,10,0,6951383569520608258,@ketikode_,ketikode,25,3420,7944,91,https://www.tiktok.com/@ketikode_/video/738384...,https://www.tiktok.com/@ketikode_


# Automate 10 pages

In [ ]:
def generate_search_id():
    timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    random_hex_length = (32-len(timestamp))//2
    random_hex = secrets.token_hex(random_hex_length).upper()
    random_id = timestamp + random_hex
    return random_id

In [ ]:
def form_api_url(page = 1, keyword = 'basang data'):
    offset = (page - 1) * 12
    params = {
        'keyword': quote(keyword),
        'offset': offset,
        'search_id': generate_search_id(),
    }
    return api_search_url + urlencode(params)

In [146]:
keyword = 'dasar python'

new_dir = re.sub(r'\W+', '_', keyword.lower())

if not os.path.exists(new_dir):
    os.makedirs(new_dir)

files = glob.glob(os.path.join(new_dir, '*'))
for f in files: os.remove(f)

start_page = 1
end_page = 10
pbar = tqdm(total=end_page-start_page+1)

li = []
for page in range(start_page, end_page+1):
    r = S.get(form_api_url(page, keyword), headers=base_headers)

    filename = os.path.join(new_dir, f'page_{page}.json')
    with open(filename, 'w') as f:
        json.dump(r.json(), f)

    pbar.update(1)

    if 'data' in r.json():
        for d in r.json()['data']:
            temp_dict = {}
            if d['type'] == 1:
                temp_dict['id'] = d['item']['id']
                temp_dict['desc'] = d['item']['desc']
                temp_dict['duration'] = d['item']['video']['duration']
                temp_dict['duration_text'] = format_timespan(d['item']['video']['duration'])
                temp_dict['play_count'] = d['item']['stats']['playCount']
                temp_dict['like_count'] = d['item']['stats']['diggCount']
                temp_dict['comment_count'] = d['item']['stats']['commentCount']
                temp_dict['bookmark_count'] = d['item']['stats']['collectCount']
                temp_dict['share_count'] = d['item']['stats']['shareCount']
                temp_dict['author_id'] = d['item']['author']['id']
                temp_dict['author_username'] = '@' + d['item']['author']['uniqueId']
                temp_dict['author_name'] = d['item']['author']['nickname']
                temp_dict['author_following_count'] = d['item']['authorStats']['followingCount']
                temp_dict['author_follower_count'] = d['item']['authorStats']['followerCount']
                temp_dict['author_like_count'] = d['item']['authorStats']['heartCount']
                temp_dict['author_post_count'] = d['item']['authorStats']['videoCount']
                temp_dict['url_video'] = 'https://www.tiktok.com/@' + d['item']['author']['uniqueId'] + '/video/' + d['item']['id']
                temp_dict['url_author'] = 'https://www.tiktok.com/@' + d['item']['author']['uniqueId']

                li.append(temp_dict)

df = pd.DataFrame(li)
df.to_excel(os.path.join(new_dir, 'data_tiktok.xlsx'), index=False)

pbar.close()

  0%|          | 0/10 [00:00<?, ?it/s]